# Complete Guide to Qiskit Primitives: Theory and Practice

This comprehensive notebook covers everything you need to know about Qiskit's **Estimator** and **Sampler** primitives, from theoretical foundations to practical implementation with advanced error mitigation techniques.

## What You'll Learn:

**Part 1: Theory**
- What the Sampler primitive does and why it exists
- Probability distributions and quasi-probabilities
- Theoretical foundations of error suppression techniques

**Part 2: Practice**
- Getting started with both Estimator and Sampler
- Setting options for error mitigation
- Implementing Dynamical Decoupling and Pauli Twirling
- Using advanced resilience options (ZNE, T-REx)
- Comparing results with and without error mitigation

---
# Part 1: Understanding the Theory
---

## 1.1 What is the Sampler Primitive?

The **Sampler** is one of the foundational primitives in Qiskit. Unlike the **Estimator**, which calculates expectation values of observables $(\langle \psi | H | \psi \rangle)$, the Sampler is designed to approximate the **quasi-probability distribution** of measurement outcomes.

In simple terms, if you run a quantum circuit that prepares a state $|\psi\rangle$, measuring it in the computational basis produces a bitstring (e.g., `101`). Running this many times (shots) gives you a histogram of counts. The Sampler formalizes this process but adds a layer of abstraction that allows for:

1. **Error Mitigation**: Automatically acting to correct or suppress errors in the readout or execution.
2. **Session Management**: Efficiently queuing multiple jobs (when using the cloud runtime).
3. **Unified Interface**: Working consistently across simulators and real hardware.

## 1.2 Probability Distributions

At its core, the Sampler estimates the probability $P(k)$ of measuring a basis state $|k\rangle$:

$$ P(k) = |\langle k | \psi \rangle|^2 $$

While a standard `backend.run()` returns raw counts (e.g., `{'00': 50, '11': 50}`), the Sampler treats these results as samples from an underlying probability distribution.

### Why "Quasi"-Probability?

You may hear the term "quasi-probability" (especially with Sampler V1 or certain error mitigation settings). This refers to distributions that might have negative values or sum to numbers other than 1 due to error mitigation post-processing (like Probabilistic Error Cancellation).

However, in standard usage with current implementations (V2) and default settings, we largely focus on **sampled bitstrings** (counts) or **probability mass functions** (normalized counts) which approximate the true probability distribution of the quantum state.

## 1.3 Theory of Error Suppression & Mitigation

One of the main reasons to use the Sampler primitive over `backend.run()` is access to advanced error handling options. Two key techniques often exposed via Sampler options are **Dynamical Decoupling** and **Pauli Twirling**.

### A. Dynamical Decoupling (DD)

**The Problem**: Qubits lose their quantum information over time due to interaction with the environment (decoherence aka $T_1$ and $T_2$ times). This happens even when the qubit is "idle" (waiting for other qubits to finish their gates).

**The Theory**: Dynamical Decoupling applies a sequence of pulses (typically $\pi$-pulses, like $X$ gates) to the idle qubits. These pulses repeatedly flip the qubit state along opposite axes on the Bloch sphere (like a "spin echo").

**Analogy**: Imagine a runner on a track. If they drift slightly left due to wind, flipping them 180 degrees makes them drift right (relative to the track), cancelling out the original drift.

**In Qiskit**: You can enable this in the Sampler options, and the transpiler/scheduler will automatically insert these sequences into idle windows.

### B. Pauli Twirling

**The Problem**: Systematic, coherent errors (e.g., a gate always over-rotating by 1 degree) are very damaging because they can add up constructively ($1 + 1 + 1...$).

**The Theory**: Twirling wraps a noisy gate (like a CNOT) with random Pauli gates ($I, X, Y, Z$) that mathematically cancel out effectively to the identity operation *if there were no noise*. However, in the presence of noise, this randomization converts "coherent" errors (systematic shifts) into "incoherent" stochastic noise (random bit flips).

**Why do this?** Incoherent noise leads to the "depolarizing channel," which is much easier to model and mitigate than coherent errors. It turns a "bad" predictable error into "random" noise that averages out.

---
# Part 2: Practical Implementation
---

## 2.1 Getting Started with Primitives (Ideal Local Simulation)

We will start by running the `Estimator` and `Sampler` on a perfect, noiseless local simulator (`AerSimulator`). This shows the basic workflow.

### Estimator Example

The **Estimator** primitive calculates expectation values of observables for given circuits. This is a fundamental task in many quantum algorithms, such as VQE.

In [1]:
# 1. Import the local simulator backend
from qiskit_aer import AerSimulator
# We'll start with an ideal (noiseless) simulator.
backend = AerSimulator()

print(f"Using local backend: {backend.name}")

Using local backend: aer_simulator


In [2]:
from qiskit.circuit.library import qaoa_ansatz
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# To make this runnable and easy to inspect, let's use a smaller 5-qubit circuit.
num_qubits = 5
entanglement = [(0, 1), (1, 2), (2, 3), (3, 4)]
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", [i, j], 0.5) for i, j in entanglement],
    num_qubits=num_qubits,
)
circuit = qaoa_ansatz(observable, reps=2)

# The parameter values now need to match the smaller circuit's parameter count
param_values = np.random.rand(circuit.num_parameters)

print(f">>> Circuit has {circuit.num_parameters} parameters.")
print(f">>> Observable Paulis: {observable.paulis}")

>>> Circuit has 4 parameters.
>>> Observable Paulis: ['IIIZZ', 'IIZZI', 'IZZII', 'ZZIII']


In [3]:
from qiskit.transpiler import generate_preset_pass_manager

# Transpiling the circuit for the backend's basis gates is good practice,
# though often optional for an ideal AerSimulator.
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
print(f">>> Circuit ops (ISA-compliant): {isa_circuit.count_ops()}")

>>> Circuit ops (ISA-compliant): OrderedDict([('rx', 10), ('rzz', 8), ('h', 5)])


In [4]:
# 2. Import the BackendEstimatorV2, which works with any local backend
from qiskit.primitives import BackendEstimatorV2 as Estimator

# Instantiate the Estimator with our local AerSimulator backend
estimator = Estimator(backend=backend)

# The run call takes a list of Primitives Unified Blocs (PUBs).
# Each PUB is a tuple of (circuit, observable, [parameter_values]).
job = estimator.run([(isa_circuit, isa_observable, param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

>>> Job ID: 390ccb0f-92ee-415c-a749-6c8ea391498b
>>> Job Status: JobStatus.DONE


In [5]:
# 3. Crucially, let's actually look at the results!
result = job.result()
print(f"\n>>> Full Result Object: {result}")

# The result for the first PUB is at index 0
pub_result = result[0]
print(f"\n  > Expectation value (EV): {pub_result.data.evs}")
print(f"  > Standard deviation: {pub_result.data.stds}")
print(f"  > Metadata: {pub_result.metadata}")


>>> Full Result Object: PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.float64(0.031193390374389167)), metadata={'target_precision': 0.015625, 'shots': 4096, 'circuit_metadata': {}})], metadata={'version': 2})

  > Expectation value (EV): 0.115478515625
  > Standard deviation: 0.031193390374389167
  > Metadata: {'target_precision': 0.015625, 'shots': 4096, 'circuit_metadata': {}}


### Sampler Example

The **Sampler** primitive generates quasi-probability distributions of bitstrings from a circuit's output. It mimics the process of measuring a quantum system multiple times.

In [6]:
from qiskit.circuit.library import efficient_su2

# Again, using a smaller circuit for clarity
sampler_circuit = efficient_su2(5, entanglement="linear", reps=2)
sampler_circuit.measure_all()
sampler_param_values = np.random.rand(sampler_circuit.num_parameters)

# We can reuse the pass manager from before to transpile
sampler_isa_circuit = pm.run(sampler_circuit)
print(f">>> Circuit ops (ISA-compliant): {sampler_isa_circuit.count_ops()}")

>>> Circuit ops (ISA-compliant): OrderedDict([('ry', 15), ('rz', 15), ('cx', 8), ('measure', 5), ('barrier', 1)])


In [7]:
# 3. Import the BackendSamplerV2
from qiskit.primitives import BackendSamplerV2 as Sampler

sampler = Sampler(backend=backend)

# A Sampler PUB is a tuple of (circuit, [parameter_values], [shots]).
job = sampler.run([(sampler_isa_circuit, sampler_param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

>>> Job ID: 47c71bd3-6098-498b-aa1e-5f3c02d7ec80
>>> Job Status: JobStatus.RUNNING


In [8]:
# 4. Get and display the results
result = job.result()
print(f"\n>>> Full Result Object: {result}")

# Get results for the first (and only) PUB
pub_result = result[0]
print(
    f"\nFirst ten results for the 'meas' output register:\n{pub_result.data.meas.get_bitstrings()[:10]}"
)


>>> Full Result Object: PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=1024, num_bits=5>)), metadata={'shots': 1024, 'circuit_metadata': {}})], metadata={'version': 2})

First ten results for the 'meas' output register:
['10000', '00000', '00000', '00001', '10111', '00001', '11001', '01010', '10001', '00000']


## 2.2 Setting Options for Error Mitigation

Now that we understand the theory and have seen basic usage, let's implement the error mitigation techniques we learned about.

Many advanced options like `resilience_level` and `dynamical_decoupling` are features of the `qiskit-ibm-runtime` service. The basic `BackendEstimatorV2` and `BackendSamplerV2` do not support them directly.

**However**, we can still test the *effect* of these options by using the **runtime primitives in local mode** with a **noisy simulator**. This allows us to learn the API and see how error mitigation works without needing a cloud account.

### Setup for Error Mitigation Examples

First, let's create a noisy backend by simulating the noise model of a real device (`FakeManilaV2`). We will use this for all the following examples. We also need to transpile our circuit for this specific noisy backend.

In [9]:
# --- ROBUST IMPORT BLOCK for Fake Backends ---
try:
    from qiskit_ibm_runtime.fake_provider import FakeManilaV2
except ImportError:
    try:
        from qiskit.providers.fake_provider import FakeManilaV2
    except ImportError:
        FakeManilaV2 = None
# ---------------------------------------------

# We need the runtime Estimator, not the backend one, to access these options
from qiskit_ibm_runtime import EstimatorV2 as RuntimeEstimator

# This cell will only run if the import was successful
if FakeManilaV2:
    # 1. Create a realistic noisy backend
    noisy_backend = AerSimulator.from_backend(FakeManilaV2())

    # 2. Transpile our circuit specifically for this noisy backend's basis gates
    pm_noisy = generate_preset_pass_manager(optimization_level=1, backend=noisy_backend)
    noisy_isa_circuit = pm_noisy.run(circuit) # Using 'circuit' from the first Estimator example
    noisy_isa_observable = observable.apply_layout(noisy_isa_circuit.layout)

    print("Noisy backend and transpiled circuit are ready for mitigation examples.")
else:
    print("Skipping error mitigation examples because FakeManilaV2 is not available.")
    # Define placeholder variables so the notebook doesn't crash
    noisy_backend = None
    noisy_isa_circuit = None
    noisy_isa_observable = None

Noisy backend and transpiled circuit are ready for mitigation examples.


### Common Sampler Options

Here we demonstrate setting options that are common to both `Sampler` and `Estimator`.

#### Option: `default_shots`

`default_shots` sets the number of shots for all runs, but can be overridden in the `run()` call. Let's demonstrate with the Sampler.

In [10]:
from qiskit.primitives import BackendSamplerV2 as Sampler

# 1. Instantiate with a default
sampler_with_options = Sampler(backend=backend, options={"default_shots": 1000})

# 2. Run a job WITHOUT specifying shots in run() - it should use the default
job1 = sampler_with_options.run([(sampler_isa_circuit, sampler_param_values)])
result1 = job1.result()
print(f"Run 1 used the default shots: {result1[0].metadata['shots']}")

# 3. Run a job WITH shots specified in run() - this should override the default
job2 = sampler_with_options.run([(sampler_isa_circuit, sampler_param_values)], shots=123)
result2 = job2.result()
print(f"Run 2 used the overridden shots: {result2[0].metadata['shots']}")

Run 1 used the default shots: 1000
Run 2 used the overridden shots: 123


## 2.3 Implementing Dynamical Decoupling

Now let's apply the **Dynamical Decoupling** theory we learned earlier. Errors on idle qubits can be canceled by adding sequences of identity operations (pulses). This is a runtime-only option.

In [11]:
if FakeManilaV2:
    # We must use the RuntimeEstimator for this option
    estimator_dd = RuntimeEstimator(mode=noisy_backend)

    # Set options for dynamic decoupling
    estimator_dd.options.dynamical_decoupling.enable = True
    estimator_dd.options.dynamical_decoupling.sequence_type = "XpXm" # A common sequence
    print(f"Dynamical Decoupling Enabled: {estimator_dd.options.dynamical_decoupling.enable}")
    print(f"Sequence Type: {estimator_dd.options.dynamical_decoupling.sequence_type}")

    # Run the job and show the result
    job_dd = estimator_dd.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_dd = job_dd.result()
    print(f"  > Expectation value (with DD): {result_dd[0].data.evs}")

Dynamical Decoupling Enabled: True
Sequence Type: XpXm
  > Expectation value (with DD): 0.125244140625


/Users/daai6ga1hou2/anaconda3/envs/qiskit_env/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:274: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XpXm'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


## 2.4 Implementing Pauli Twirling

**Pauli Twirling** involves sandwiching gates with random Pauli operators to average out certain types of coherent noise. This is a runtime `Estimator` option.

In [12]:
if FakeManilaV2:
    estimator_twirl = RuntimeEstimator(mode=noisy_backend)
    
    # Set twirling options
    estimator_twirl.options.twirling.enable_gates = True
    estimator_twirl.options.twirling.num_randomizations = 16
    print(f"Gate Twirling Enabled: {estimator_twirl.options.twirling.enable_gates}")
    
    job_twirl = estimator_twirl.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_twirl = job_twirl.result()
    print(f"  > Expectation value (with Twirling): {result_twirl[0].data.evs}")

Gate Twirling Enabled: True
  > Expectation value (with Twirling): 0.1044921875


/Users/daai6ga1hou2/anaconda3/envs/qiskit_env/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:274: UserWarning: Options {'twirling': {'enable_gates': True, 'num_randomizations': 16}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


## 2.5 Advanced Error Mitigation: Resilience Options

`resilience` options are a powerful feature of the runtime `Estimator` for reducing the impact of noise on results.

**Note on UserWarnings:** When running locally, you might see a warning that `resilience_level` has no effect. However, the underlying `AerSimulator` *does* perform the mitigation, and you will see the results improve.

### Zero Noise Extrapolation (ZNE)

This technique runs the circuit at different amplified noise levels and extrapolates the result back to the zero-noise limit.

In [13]:
if FakeManilaV2:
    estimator_zne = RuntimeEstimator(mode=noisy_backend)
    
    # Set options for ZNE (level 0 disables presets)
    estimator_zne.options.resilience_level = 0
    estimator_zne.options.resilience.zne_mitigation = True
    print(f"ZNE Enabled: {estimator_zne.options.resilience.zne_mitigation}")

    job_zne = estimator_zne.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_zne = job_zne.result()
    print(f"  > Expectation value (with ZNE): {result_zne[0].data.evs}")

ZNE Enabled: True
  > Expectation value (with ZNE): 0.095703125


/Users/daai6ga1hou2/anaconda3/envs/qiskit_env/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:190: UserWarning: The resilience_level option has no effect in local testing mode.
  warnings.warn("The resilience_level option has no effect in local testing mode.")
/Users/daai6ga1hou2/anaconda3/envs/qiskit_env/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:274: UserWarning: Options {'resilience': {'zne_mitigation': True}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


### Twirling Readout Error Extinction (T-REx)

This method specifically targets errors that occur during the final measurement of the qubits.

In [14]:
if FakeManilaV2:
    estimator_trex = RuntimeEstimator(mode=noisy_backend)
    
    # Set options for T-REx (level 0 disables presets)
    estimator_trex.options.resilience_level = 0
    estimator_trex.options.resilience.measure_mitigation = True
    print(f"Measurement Mitigation (T-REx) Enabled: {estimator_trex.options.resilience.measure_mitigation}")
    
    job_trex = estimator_trex.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_trex = job_trex.result()
    print(f"  > Expectation value (with T-REx): {result_trex[0].data.evs}")

Measurement Mitigation (T-REx) Enabled: True


/Users/daai6ga1hou2/anaconda3/envs/qiskit_env/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:274: UserWarning: Options {'resilience': {'measure_mitigation': True}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


  > Expectation value (with T-REx): 0.118896484375


## 2.6 Final Comparison: Using `resilience_level` Presets

The `resilience_level` is a convenient preset that combines multiple error mitigation techniques.

*   **Level 0:** No mitigation.
*   **Level 1:** ZNE + T-REx. This is a good general-purpose starting point.

Let's compare the results of running with no mitigation vs. `resilience_level=1` against the true ideal value.

In [15]:
if FakeManilaV2:
    # --- Ideal (noiseless) result for a baseline ---
    ideal_backend = AerSimulator()
    estimator_ideal = RuntimeEstimator(mode=ideal_backend)
    result_ideal = estimator_ideal.run([(isa_circuit, isa_observable, param_values)]).result()

    # --- Estimator WITHOUT any mitigation ---
    estimator_no_mitigation = RuntimeEstimator(mode=noisy_backend)
    estimator_no_mitigation.options.resilience_level = 0
    job_no_mitigation = estimator_no_mitigation.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_no_mitigation = job_no_mitigation.result()

    # --- Estimator WITH Level 1 mitigation ---
    estimator_with_mitigation = RuntimeEstimator(mode=noisy_backend)
    estimator_with_mitigation.options.resilience_level = 1
    job_with_mitigation = estimator_with_mitigation.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_with_mitigation = job_with_mitigation.result()

    print("\n--- FINAL COMPARISON ---")
    print(f"  > Expectation value (Ideal, Noiseless): {result_ideal[0].data.evs}")
    print(f"  > Expectation value (Noisy, NO mitigation): {result_no_mitigation[0].data.evs}")
    print(f"  > Expectation value (Noisy, WITH mitigation, resilience_level=1): {result_with_mitigation[0].data.evs}")


--- FINAL COMPARISON ---
  > Expectation value (Ideal, Noiseless): 0.13037109375
  > Expectation value (Noisy, NO mitigation): 0.140380859375
  > Expectation value (Noisy, WITH mitigation, resilience_level=1): 0.105224609375


---
# Summary
---

In this comprehensive notebook, we covered:

## Theory:
1. **Sampler Purpose**: We learned it samples the output probability distribution $(|\langle k | \psi \rangle|^2$).
2. **Dynamical Decoupling Theory**: How refocusing idle qubits with pulse sequences cancels decoherence.
3. **Pauli Twirling Theory**: How randomizing coherent errors converts them to incoherent stochastic noise.

## Practice:
1. **Basic Primitives**: Set up and ran both Estimator and Sampler on ideal simulators.
2. **Options Configuration**: Learned to set `default_shots` and other options.
3. **Error Mitigation Implementation**:
   - Dynamical Decoupling (DD)
   - Pauli Twirling
   - Zero Noise Extrapolation (ZNE)
   - Twirling Readout Error Extinction (T-REx)
4. **Resilience Levels**: Compared results with and without error mitigation.

You now have both the theoretical foundation and practical skills to use Qiskit primitives effectively!

## Practice Questions

**1. What is the primary output calculated by the Sampler primitive?**

A) Expectation values of observables

B) Quasi-probability distributions of measurement outcomes

C) The statevector of the quantum system

D) The unitary matrix of the circuit

***Answer:***
<Details>
<br/>
B) Quasi-probability distributions of measurement outcomes
</Details>

---

**2. Which error mitigation technique suppresses decoherence errors on idle qubits by applying a sequence of pulses (like X gates)?**

A) Pauli Twirling

B) Zero Noise Extrapolation (ZNE)

C) Dynamical Decoupling (DD)

D) Probabilistic Error Cancellation (PEC)

***Answer:***
<Details>
<br/>
C) Dynamical Decoupling (DD)
</Details>

---

**3. In the Qiskit Runtime Estimator V2, how would you explicitly enable Zero Noise Extrapolation (ZNE) if you are not using a resilience level preset?**

A) `options.resilience.zne_mitigation = True`

B) `options.zne = True`

C) `options.error_mitigation.zne = True`

D) `options.resilience_level = 'ZNE'`

***Answer:***
<Details>
<br/>
A) `options.resilience.zne_mitigation = True`
</Details>